In [2]:
%%writefile 'streamlit-app.py'

import pandas as pd
import streamlit as st

import datetime 
import requests
import json

date_str = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
parking_str='Riva Reno'
API_URL = f'https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/disponibilita-parcheggi-storico/records?where=data%3C%3D%27{date_str}%27%20and%20parcheggio%3D%27{parking_str}%27&order_by=data%20DESC&limit=100'

SERVICE_URL = 's-python-python-serve-a101972d-4860-4ae5-b6c0-83da27fe5bda.digitalhub-tenant1.svc.cluster.local:8080'


latest_data_file = 'last_records.json'

with requests.get(API_URL) as r:
    with open(latest_data_file, "wb") as f:
        f.write(r.content)

with open(latest_data_file) as f:
    json_data = json.load(f)
    df_latest = pd.json_normalize(json_data['results']).drop(columns=['guid', 'occupazione']).rename(columns={"coordinate.lon": "lon", "coordinate.lat": "lat"})
    df_latest.data = df_latest.data.astype('datetime64')
    df_latest['value'] = df_latest.posti_occupati / df_latest.posti_totali
    df_latest['date'] = df_latest.data.dt.round('30min')
    df_latest = df_latest.drop(columns=['parcheggio'])
    df_latest = df_latest.groupby('date').agg({'value': 'mean'})


jsonstr = df_latest.reset_index().to_json(orient='records')
arr = json.loads(jsonstr)

with requests.post(f'http://{SERVICE_URL}', json={"inference_input":arr}) as r:
    res = r.json()
print(res)

rdf = pd.read_json(json.dumps(res), orient="records")

st.write("""prediction""")
st.line_chart(rdf, x="date", y="value")

Overwriting streamlit-app.py


In [3]:
!pip install streamlit


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!streamlit run streamlit-app.py --browser.gatherUsageStats false


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.4.105:8501
  External URL: http://172.213.198.128:8501

[{'date': 1721395800000, 'value': 0.1980021685}, {'date': 1721397600000, 'value': 0.1990738656}, {'date': 1721399400000, 'value': 0.1670521677}, {'date': 1721401200000, 'value': 0.1395947425}, {'date': 1721403000000, 'value': 0.1597397088}, {'date': 1721404800000, 'value': 0.1490922439}, {'date': 1721406600000, 'value': 0.1620526471}, {'date': 1721408400000, 'value': 0.1405377663}, {'date': 1721410200000, 'value': 0.1460526221}, {'date': 1721412000000, 'value': 0.1319060251}, {'date': 1721413800000, 'value': 0.1284732108}, {'date': 1721415600000, 'value': 0.1217897569}, {'date': 1721417400000, 'value': 0.061027846}, {'date': 1721419200000, 'value': 0.072659693}, {'date': 1721421000000, 'value': 0.0640241991}, {'date': 1721422800000, 'value': 0.0532764625}, {'date': 1721424600000, 'value': 0.0960479274}, {'date'